We plot the data contained in a ARGO profilers measuring biological parameters

In [1]:
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
import glob
import os
import seawater
import cmocean
from matplotlib import rcParams
from mpl_toolkits.basemap import Basemap
%matplotlib inline

In [40]:
rcParams.update({'font.size': 16})
figdir = "/home/ctroupin/Projects2/201501_InsTAC/Graphical _Material/2016_Q2/"
datadir = "/data_local/DataOceano/CMEMS/INSITU_GLO_NRT_OBSERVATIONS_013_030/history/profiler-glider"
dlon, dlat = 2., 1.

In [54]:
datafilelist = sorted(glob.glob(os.path.join(datadir, '*.nc')))
for datafiles in datafilelist:
    print('Working on %s' %(datafiles))
    figbasename = os.path.basename(datafiles)[:-3]
          
    # Read the variables from the data file
    time, time_units, lon, lat, pressure, temp, temp_qc, psal, psal_qc, chloro, chloro_qc = read_variables(datafiles)
    
    # Convert time to dates and change the vector to a 2D array
    datesarray = convert_time(time, time_units)
    
    # Compute initial and final dates
    dayinit, dayend = get_init_final_dates(datesarray)
    
    # Apply QC
    temp, psal, chloro = apply_qc_variables(temp, temp_qc, psal, psal_qc, chloro, chloro_qc)
    
    # Convert pressure to depth using [Soaunders (1981)] formula
    depth = seawater.eos80.dpth(pressure, np.tile(lat, (pressure.shape[1], 1)).T)
    
    # Plots:
    # -----------------------------
    
    depthmin, depthmax = 0.0, 1000.0

    # Temperature
    varname = 'temperature'
    tmin, tmax = 4.0, 15.0
    cmap = cmocean.cm.temperature
    var_units = '$^{\circ}$C'
    extend_type = 'both'
    #plot_variable_time_depth(varname, temp, cmap, var_units, tmin, tmax, depthmin, depthmax, extend_type, figbasename)
          
    # Salinity
    varname = 'salinity'
    smin, smax = 34.5, 35.5
    cmap = cmocean.cm.salinity
    var_units = ' '
    extend_type = 'both'
    #plot_variable_time_depth(varname, psal, cmap, var_units, smin, smax, depthmin, depthmax, extend_type, figbasename)
    
    # Chlorophyll concentration
    varname = 'chlorophyll'
    cmap = cmocean.cm.chlorophyll
    var_units = 'mg/m$^3$'
    chloro_min, chloro_max = 0, 3.0
    extend_type = 'max'
    #plot_variable_time_depth(varname, chloro, cmap, var_units, chloro_min, chloro_max, depthmin, 
    #                         depthmax, extend_type, figbasename)

    
    # T-S diagram
    density, density2plot = compute_density()
    plot_TS(temp, psal, depth)
    
    # Trajectory of the profiler
    #plot_trajectory(lon, lat, dlon, dlat, figbasename, dayinit, dayend)

Working on /data_local/DataOceano/CMEMS/INSITU_GLO_NRT_OBSERVATIONS_013_030/history/profiler-glider/GL_PR_PF_6901646.nc
Working on /data_local/DataOceano/CMEMS/INSITU_GLO_NRT_OBSERVATIONS_013_030/history/profiler-glider/GL_PR_PF_6901647.nc
Working on /data_local/DataOceano/CMEMS/INSITU_GLO_NRT_OBSERVATIONS_013_030/history/profiler-glider/GL_PR_PF_6901648.nc
Working on /data_local/DataOceano/CMEMS/INSITU_GLO_NRT_OBSERVATIONS_013_030/history/profiler-glider/GL_PR_PF_6901649.nc
Working on /data_local/DataOceano/CMEMS/INSITU_GLO_NRT_OBSERVATIONS_013_030/history/profiler-glider/GL_PR_PF_6901651.nc
Working on /data_local/DataOceano/CMEMS/INSITU_GLO_NRT_OBSERVATIONS_013_030/history/profiler-glider/GL_PR_PF_6901653.nc


In [43]:
def read_variables(datafiles):
    with netCDF4.Dataset(datafiles) as nc:
        time = nc.variables['TIME'][:]
        time_units = nc.variables['TIME'].units
        lon = nc.variables['LONGITUDE'][:]
        lat = nc.variables['LATITUDE'][:]
        pressure = nc.variables['PRES'][:]
        temp = nc.variables['TEMP'][:]
        temp_qc = nc.variables['TEMP_QC'][:]
        psal = nc.variables['PSAL'][:]
        psal_qc = nc.variables['PSAL_QC'][:]
        chloro = nc.variables['CPHL_ADJUSTED'][:]
        chloro_qc = nc.variables['CPHL_ADJUSTED_QC'][:]
        chloro_units = nc.variables['CPHL'].units
    return time, time_units, lon, lat, pressure, temp, temp_qc, psal, psal_qc, chloro, chloro_qc

In [3]:
def convert_time(time, time_units):
    '''Convert time to dates and change the vector to a 2D array.'''
    dates = netCDF4.num2date(time, time_units)
    datesarray = np.tile(dates, (pressure.shape[1], 1)).T
    return datesarray

In [47]:
def get_init_final_dates(datesarray):
    '''Compute initial and final dates'''
    dayinit = datesarray[0, 0].strftime('%d %B %Y')
    dayend = datesarray[-1, 0].strftime('%d %B %Y')
    return dayinit, dayend

In [42]:
def apply_qc_variables(temp, temp_qc, psal, psal_qc, chloro, chloro_qc):
    '''Apply the quality flags (we keep only the good values'''
    temp = np.ma.masked_where(temp_qc != 1, temp)
    psal = np.ma.masked_where(psal_qc != 1, psal)
    # chloro = np.ma.masked_where(chloro_qc > 3., chloro)
    return temp, psal, chloro

In [49]:
def plot_variable_time_depth(varname, var2plot, cmap, var_units, var_min, 
                             var_max, depthmin, depthmax, extendtype, figbasename):
    figname = figbasename + '_' + varname
    fig = plt.figure(figsize=(15, 10))
    plt.scatter(datesarray, pressure, c=var2plot, s=7, 
                edgecolor='None', cmap=cmap)
    plt.xlabel('Time', fontsize=18)
    plt.ylabel('Depth\n(m)', fontsize=18, rotation=0, ha='right')
    cbar = plt.colorbar(extend='max')
    cbar.set_label(var_units, rotation=0, ha='left')
    # cbar.set_clim(var_min, var_max)
    ax = plt.gca()
    ax.set_ylim(depthmin, depthmax)
    ax.invert_yaxis()
    fig.autofmt_xdate()
    plt.savefig(os.path.join(figdir, figname), dpi=300)
    plt.close()

In [53]:
def plot_TS(temp, psal, depth):
    figname = figbasename + '_TSdiagram'
    
    tmin = np.floor(temp.min())
    tmax = np.ceil(temp.max())
    smin = np.floor(psal.min())
    smax = np.ceil(psal.max())
    
    # Compute density for different T-S pairs
    density2plot = np.arange(24, 30, .5)
    ds, dt = 0.05, 0.1
    tvec = np.arange(tmin, tmax, dt)
    svec = np.arange(smin, smax, ds)
    ssvec, ttvec = np.meshgrid(svec, tvec)
    density = seawater.eos80.dens0(ssvec, ttvec) - 1000.0
    
    # Make the plot
    fig = plt.figure(figsize=(15, 15))
    plt.scatter(psal, temp, s=5, c=depth,
               edgecolor='None', cmap=cmocean.cm.swh)
    cbar = plt.colorbar()
    plt.xlabel('Salinity', fontsize=18)
    plt.ylabel('Temperature\n($^{\circ}$C)', rotation=0, ha='right', fontsize=18)
    cont = plt.contour(svec, tvec, density, levels=density2plot, colors='.65', linestyles='dashed')
    plt.clabel(cont,inline=True, fmt='%1.1f')
    #plt.xlim(smin, smax)
    #plt.ylim(tmin, tmax)
    cbar.set_label('Depth\n(m)', rotation=0, ha='left')
    plt.savefig(os.path.join(figdir, figname), dpi=300)
    plt.close()

In [46]:
def plot_trajectory(lon, lat, dlon, dlat, figbasename, dayinit, dayend):
    
    coordinates = np.floor(lon.min()), np.ceil(lon.max()), np.floor(lat.min()), np.ceil(lat.max())
    
    m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
                    urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
                    lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')
    figname = figbasename + '_trajectory'
    fig = plt.figure(figsize=(12, 12))

    lonproj, latproj = m(lon, lat)
    m.plot(lonproj, latproj, 'ko-')
    m.plot(lonproj[0], latproj[0], '>g', markersize=15, zorder=2, label='Starting point')
    m.plot(lonproj[-1], latproj[-1], 'sr', markersize=15, zorder=2, label='Final point')
    m.drawparallels(np.arange(round(coordinates[2]), coordinates[3], dlat), linewidth=0.2,
                    labels=[1, 0, 0, 0], fontname='Times New Roman', fontsize=16, zorder=1)
    m.drawmeridians(np.arange(round(coordinates[0]), coordinates[1], dlon), linewidth=0.2,
                    labels=[0, 0, 0, 1], fontname='Times New Roman', fontsize=16, zorder=1)
    m.drawcoastlines()
    plt.legend(numpoints=1, fontsize=16, loc=2)
    plt.title('Trajectory from %s to %s' %(dayinit, dayend), fontsize=24)
    plt.savefig(os.path.join(figdir, figname), dpi=300)
    plt.close()